# Initialization

In [1]:
# data processing
import numpy as np
import pandas as pd 
from collections import defaultdict

# data visualization
import seaborn as sns
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib import style
sns.set()
import urllib.request


import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))
from tensorflow import keras

# Notebook auto reloads code. (Ref: http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython)
%load_ext autoreload
%autoreload 2

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
# If using Google Colab
from google.colab import drive
BASE_PATH = '/content/drive'
drive.mount(BASE_PATH)

# change directory
import os
PROJECT_PATH = os.path.join(BASE_PATH, "MyDrive", "2021-09 Fall Semester", "ECBM 4040 Neural Network Deep Learning", "Project", "recreating-residual-attention-network")
os.chdir(PROJECT_PATH)

Mounted at /content/drive


In [3]:
# Import created modules
from src.models.ResidualAttentionNetwork import ResidualAttentionNetwork, Attention56, Attention92, Attention128, Attention164
from src.utils import generate_data

# Modelling

In [4]:
INPUT_SHAPE, NUM_CLASS, train_ds, val_ds, test_ds, _ = generate_data.get_cifar10()

170508288/170498071 [==============================] - 11s 0us/step


In [5]:
BATCH_SIZE = 256
N_EPOCH = 10


In [6]:
train_ds = train_ds.shuffle(buffer_size=BATCH_SIZE*4).batch(BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.shuffle(buffer_size=BATCH_SIZE*4).batch(BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.shuffle(buffer_size=BATCH_SIZE*4).batch(BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)

## Naive Attention Learning vs Residual Attention Learning

### Naive Attention Learning

In [7]:
ran_model = ResidualAttentionNetwork(input_shape=INPUT_SHAPE, num_class=NUM_CLASS, learning_type='nal')
inputs = tf.keras.Input((32, 32, 3))
ran_model(inputs)
ran_model.summary()

Model: "residual_attention_network"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (None, 32, 32, 3)         0         
                                                                 
 conv2d (Conv2D)             multiple                  864       
                                                                 
 batch_normalization (BatchN  multiple                 128       
 ormalization)                                                   
                                                                 
 re_lu (ReLU)                multiple                  0         
                                                                 
 max_pooling2d (MaxPooling2D  multiple                 0         
 )                                                               
                                                                 
 residual_unit (ResidualUnit  multiple  

In [8]:
nal_model = ran_model

nal_model.compile(optimizer=tf.keras.optimizers.Adam(), 
                  loss=tf.keras.losses.CategoricalCrossentropy(), 
                  metrics=['accuracy'])


history = nal_model.fit(train_ds,
                        validation_data=val_ds,
                        epochs=N_EPOCH, verbose=1)

Epoch 1/10
157/157 [==============================] - 75s 315ms/step - loss: 2.4122 - accuracy: 0.1538 - val_loss: 2.4336 - val_accuracy: 0.0839
Epoch 2/10
157/157 [==============================] - 45s 288ms/step - loss: 2.1877 - accuracy: 0.2224 - val_loss: 2.7091 - val_accuracy: 0.1459
Epoch 3/10
157/157 [==============================] - 45s 286ms/step - loss: 2.0631 - accuracy: 0.2672 - val_loss: 2.3955 - val_accuracy: 0.2169
Epoch 4/10
157/157 [==============================] - 45s 288ms/step - loss: 1.9851 - accuracy: 0.2907 - val_loss: 1.9223 - val_accuracy: 0.3110
Epoch 5/10
157/157 [==============================] - 45s 289ms/step - loss: 1.9185 - accuracy: 0.3146 - val_loss: 1.9016 - val_accuracy: 0.3278
Epoch 6/10
157/157 [==============================] - 45s 289ms/step - loss: 1.8648 - accuracy: 0.3274 - val_loss: 1.9299 - val_accuracy: 0.3428
Epoch 7/10
157/157 [==============================] - 45s 288ms/step - loss: 1.8251 - accuracy: 0.3427 - val_loss: 1.8776 - val_ac

### Residual Attention Learning

In [9]:
ran_model = ResidualAttentionNetwork(input_shape=INPUT_SHAPE, num_class=NUM_CLASS, learning_type='arl')
inputs = tf.keras.Input((32, 32, 3))
ran_model(inputs)
ran_model.summary()

Model: "residual_attention_network_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (None, 32, 32, 3)         0         
                                                                 
 conv2d_139 (Conv2D)         multiple                  864       
                                                                 
 batch_normalization_107 (Ba  multiple                 128       
 tchNormalization)                                               
                                                                 
 re_lu_107 (ReLU)            multiple                  0         
                                                                 
 max_pooling2d_7 (MaxPooling  multiple                 0         
 2D)                                                             
                                                                 
 residual_unit_33 (ResidualU  multiple

In [10]:
arl_model = ran_model

arl_model.compile(optimizer=tf.keras.optimizers.Adam(), 
                  loss=tf.keras.losses.CategoricalCrossentropy(), 
                  metrics=['accuracy'])


history = arl_model.fit(train_ds,
                        validation_data=val_ds,
                        epochs=N_EPOCH, verbose=1)

Epoch 1/10
157/157 [==============================] - 64s 306ms/step - loss: 2.4113 - accuracy: 0.1523 - val_loss: 2.5067 - val_accuracy: 0.1000
Epoch 2/10
157/157 [==============================] - 45s 288ms/step - loss: 2.1210 - accuracy: 0.2495 - val_loss: 2.5251 - val_accuracy: 0.1268
Epoch 3/10
157/157 [==============================] - 45s 288ms/step - loss: 1.9903 - accuracy: 0.2894 - val_loss: 2.1483 - val_accuracy: 0.2473
Epoch 4/10
157/157 [==============================] - 46s 290ms/step - loss: 1.8977 - accuracy: 0.3189 - val_loss: 1.9694 - val_accuracy: 0.2904
Epoch 5/10
157/157 [==============================] - 45s 289ms/step - loss: 1.8269 - accuracy: 0.3391 - val_loss: 1.7982 - val_accuracy: 0.3653
Epoch 6/10
157/157 [==============================] - 45s 288ms/step - loss: 1.7730 - accuracy: 0.3625 - val_loss: 1.7727 - val_accuracy: 0.3656
Epoch 7/10
157/157 [==============================] - 45s 289ms/step - loss: 1.7417 - accuracy: 0.3726 - val_loss: 1.7011 - val_ac

## Different number of attention module

In this part, the experiment is done using Residual Attention Learning mechanism

### Attention-56

In [11]:
ran_model = Attention56()
inputs = tf.keras.Input((32, 32, 3))
ran_model(inputs)
ran_model.summary()

Model: "attention56"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_1 (Sequential)   (None, 32, 32, 3)         0         
                                                                 
 conv2d_278 (Conv2D)         multiple                  864       
                                                                 
 batch_normalization_214 (Ba  multiple                 128       
 tchNormalization)                                               
                                                                 
 re_lu_214 (ReLU)            multiple                  0         
                                                                 
 max_pooling2d_14 (MaxPoolin  multiple                 0         
 g2D)                                                            
                                                                 
 residual_unit_66 (ResidualU  multiple                 

In [ ]:
a56_model = ran_model

a56_model.compile(optimizer=tf.keras.optimizers.Adam(), 
                  loss=tf.keras.losses.CategoricalCrossentropy(), 
                  metrics=['accuracy'])


history = a56_model.fit(train_ds,
                        validation_data=val_ds,
                        epochs=N_EPOCH, verbose=1)

Epoch 1/10
157/157 [==============================] - 235s 1s/step - loss: 2.5170 - accuracy: 0.1221 - val_loss: 2.4114 - val_accuracy: 0.1000
Epoch 2/10
  8/157 [>.............................] - ETA: 3:00 - loss: 2.3873 - accuracy: 0.1514

KeyboardInterrupt: ignored

### Attention-92

In [12]:
ran_model = Attention92()
inputs = tf.keras.Input((32, 32, 3))
ran_model(inputs)
ran_model.summary()

Model: "attention92"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_2 (Sequential)   (None, 32, 32, 3)         0         
                                                                 
 conv2d_417 (Conv2D)         multiple                  864       
                                                                 
 batch_normalization_321 (Ba  multiple                 128       
 tchNormalization)                                               
                                                                 
 re_lu_321 (ReLU)            multiple                  0         
                                                                 
 max_pooling2d_21 (MaxPoolin  multiple                 0         
 g2D)                                                            
                                                                 
 residual_unit_99 (ResidualU  multiple                 

In [ ]:
a92_model = ran_model

a92_model.compile(optimizer=tf.keras.optimizers.Adam(), 
                  loss=tf.keras.losses.CategoricalCrossentropy(), 
                  metrics=['accuracy'])


history = a92_model.fit(train_ds,
                        validation_data=val_ds,
                        epochs=N_EPOCH, verbose=1)

### Attention-128

In [13]:
ran_model = Attention128()
inputs = tf.keras.Input((32, 32, 3))
ran_model(inputs)
ran_model.summary()

Model: "attention128"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_3 (Sequential)   (None, 32, 32, 3)         0         
                                                                 
 conv2d_670 (Conv2D)         multiple                  864       
                                                                 
 batch_normalization_515 (Ba  multiple                 128       
 tchNormalization)                                               
                                                                 
 re_lu_515 (ReLU)            multiple                  0         
                                                                 
 max_pooling2d_34 (MaxPoolin  multiple                 0         
 g2D)                                                            
                                                                 
 residual_unit_159 (Residual  multiple                

In [ ]:
a128_model = ran_model

a128_model.compile(optimizer=tf.keras.optimizers.Adam(), 
                   loss=tf.keras.losses.CategoricalCrossentropy(), 
                   metrics=['accuracy'])


history = a128_model.fit(train_ds,
                         validation_data=val_ds,
                         epochs=N_EPOCH, verbose=1)

### Attention-164

In [14]:
ran_model = Attention164()
inputs = tf.keras.Input((32, 32, 3))
ran_model(inputs)
ran_model.summary()

Model: "attention164"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_4 (Sequential)   (None, 32, 32, 3)         0         
                                                                 
 conv2d_1037 (Conv2D)        multiple                  864       
                                                                 
 batch_normalization_796 (Ba  multiple                 128       
 tchNormalization)                                               
                                                                 
 re_lu_796 (ReLU)            multiple                  0         
                                                                 
 max_pooling2d_53 (MaxPoolin  multiple                 0         
 g2D)                                                            
                                                                 
 residual_unit_246 (Residual  multiple                

In [ ]:
a164_model = ran_model

a164_model.compile(optimizer=tf.keras.optimizers.Adam(), 
                   loss=tf.keras.losses.CategoricalCrossentropy(), 
                   metrics=['accuracy'])


history = a164_model.fit(train_ds,
                         validation_data=val_ds,
                         epochs=N_EPOCH, verbose=1)